In [ ]:
import numpy as np

eigvec = np.load("pbe/mode-1.npy")

In [ ]:
import matplotlib.pyplot as plt

from mlip_arena.models import MLIPEnum

npoints = 101
max_disp = 0.5
disps = np.linspace(-max_disp, max_disp, npoints)

replicas = (2, 2, 2)

models = ["MACE-MP(M)", "MatterSim", "ORBv2", "SevenNet", "CHGNet", "M3GNet"]

fig, axes = plt.subplots(
    figsize=(6, 4),
    nrows=2,
    ncols=len(models) // 2,
    constrained_layout=True,
)

axes = axes.flatten()

i = 0

for model in MLIPEnum:
    if model.name not in models:
        continue

    calc = get_calculator(model, device="cuda")

    emin = float("inf")
    for a in [3.7, 3.85, 4.0, 4.15]:  # [3.8, 3.9, 4.0, 4.1]
        atoms = read("BZO_cubic_prim.xyz")
        atoms.set_cell(cell=[a, a, a], scale_atoms=True)
        # atoms = atoms * replicas

        energies = []

        for disp in disps:
            atoms_disp = atoms.copy()
            atoms_disp.calc = calc

            atoms_disp.positions += eigvec * disp

            energy = atoms_disp.get_potential_energy() / len(atoms_disp)
            energies.append(energy)

        energies = np.array(energies)

        # shift the middle to 0
        energies -= energies[npoints // 2]
        emin = min(emin, energies.min())

        axes[i].plot(disps, energies * 1000, label=f"{a:.2f} Å")

    axes[i].axhline(0, color="black", alpha=0.1)
    emin -= 1e-4
    axes[i].set(
        title=model.name,
        xlabel="Displacement (Å)",
        ylabel="Energy (meV/atom)",
        # xlim=(-0.1, 0.1),
        # ylim=(-0.1, 0.5),
    )

    i += 1
    # break

fig.legend(
    axes[0].get_legend_handles_labels()[0],
    axes[0].get_legend_handles_labels()[1],
    loc="lower center",
    bbox_to_anchor=(0.5, 1),
    ncol=len(axes[0].get_legend_handles_labels()[0]),
    # fontsize=6
)
plt.show()
